# The Problem: Extracting neural activity from raw data

## Description

As neuroscientists, we are broadly interested in how groups of neurons coordinate their patterns of activity to produce computations. Due to advances in microscopy and molecular engineering, we can directly observe the firing of individual neurons reported by a fluorescent marker. The raw data we then work with is a miscroscopic video of a brain region with many neurons dynamically changing their brightness as they activate and inactivate. For this exercise we will focus on a small cropped patch of the video and play with some core methods used for analyzing this type of data.

Our goal is to extract the activity of the individual neurons from this video. Therefore, we need to go from the provided video to a matrix of dimensions: `NxT` ('neurons' x 'time'), where the value of each entry is the fluorescence amplitude of a given neuron `n` at a given timepoint `t`.

If you look in the literature, you'll find many methods for performing this type of analysis. We will apply simplified versions of modern methods. While all problems are useful for us, we really hope that you find time to put some effort and thought into problem 3. The analysis pipeline will be as follows:
1. Motion correction
2. Compare methods for blind source separation
3. Run a hyperparameter sweep using NMF 


## What we're looking for

For each question, please answer with both an explanation in words and solution in code:
- **code**: Can either be pseudocode (at any level of abstraction) or real code. Real code answers should ideally be in Python or MATLAB, but other languages are acceptable but those are just the ones we are most familiar with.
- **words**: Ideally a few sentences demonstrating your perspective on the problem. Show us how you think and talk through problems. Answers should be clear and concise, interpretable by a scientist with some programming experience.

Your solutions should demonstrate understanding of:
- **The biological problem and its significance**
    - The underlying biology, data acquisition method, and their impact on the analysis chosen
- **The data pipeline as a whole**
    - How would the output look different if any one of the intermediate steps were omitted?
    - Which parameters can be hard coded, which must be user specified, and which must be adaptive?
- **Basic signal processing and image processing**
    - Ability to appropriately use different rolling statistics
    - Ability to select appropriate heuristics
- **Unsupervised machine learning methods**
    - Blind Source Separation Methods and their alternatives 
    - Parameter estimation
    - Ideas to improve performance or efficiency

## Problem 0: getting the data onto your computer

The data is stored as a Tiff file. You can download it here (https://www.dropbox.com/s/7h44is0105o1m5w/activity_video_crop.tif?dl=0). 
Import the data into your workspace and make it a 3D array where the first dimesion is time `T`, the second dimension is frame height `y`, and the third dimension is frame width `x`. You can use the below cell to visualize the video (or you can download a free software called FIJI https://imagej.net/Fiji for a more full fledged suite of microscopy visualization tools)

In [1]:
import numpy as np
from matplotlib import pyplot as plt

import matplotlib.animation
from IPython.display import HTML

def plot_video(images, start, nframes, frameRate):
    """
    Simple video plotter
    
    Args:
        images (np.ndarray): array (shape frames x height x width)
        start (int): index of the start of the video
        nframes (int): number of frames to display
        frameRate (int): numbner of frames per second (fps)
    
    Returns:
        ani (FuncAnimation): animator function object, displays video when "to_html5_video()" called
    """
    fig = plt.figure()
    im = plt.imshow(images[start,...], 
                    vmin=np.min(images[start:(start+nframes),...]), 
                    vmax=np.percentile(images[start:(start+nframes),...], 99.99))
    plt.axis('off')
    
    def animate(n): 
        im.set_data(images[start+n,...])
        
    return matplotlib.animation.FuncAnimation(fig, animate, frames=nframes, interval = round((1/frameRate)*1000))

In [ ]:
from tifffile import imread

frames = imread('/media/rich/bigSSD RH/downloads_tmp/activity_video_crop.tif')
ani = plot_video(frames, 2500, 3000, 120)
HTML(ani.to_html5_video())

## Problem 1: Motion correction

**Note: Try to get through this section quickly, since the next problems are more interesting, informative for us, and challenging.**

The video given to you is a small region (about 60 x 60 microns) of the brain. It was recorded from a mouse while it was headfixed to a 2-photon microscope. The mouse was on a treadmill, and, when it ran, a small amount of movement (on the order of about 3-15 microns, which translates to a few pixels) occured in the brain. This movement can be seen by eye if the movie if played back. 

You are tasked with conceiving of and/or implementing a method for doing motion correction in this video. Although motion correction can be defined in many ways, for our purposes, you should:
- Ensure the same fixed set of pixels in each frame correspond to a single neuron
- Maximize the number of pixels that each neuron occupies 

The resulting video should be exactly like the original, just with the motion removed. The method you construct should have the following attributes:
- Offsets should make sense: In our implemention, we found it necessary to clean up our results by filtering the x and y offsets in order for them to make sense, physiologically. In this context, 'makes sense' means that the offsets should be biologically plausible (ie., no sudden jumps).
- The corrected video should be cropped: Motion correction generally causes a corrected image to have blank space between the edge of the frame and the edge of the shifted image. The size of this blank space will change from to frame as the offsets change. The full corrected video should be cropped such that there is never any blank space near the edges. To accomplish this, simply crop the new video to include only those pixels that never become blank.
- We recommend only using 2D translational offsets: Non-rigid registration, local flow fields, and even affine transforms are unnecessary and not biologically reasonable here.
- For simplicity, we recommend not using subpixel registration. It can yield improved results, but cause difficulties in subsequent analyses.

Some hints:
- The video is very noisy. For some methods, it is useful to do a rolling average (or some kind of rolling statistic) on frames nearby in time. We recommend looking at the timescale of the motion artifacts to determine how many frames to include in such a filter.
- If you plan on using a template/reference-image based method for alignment, we recommend creating a high quality template. The average image of the video (averaged in time) works okay to get a rough estimate of the motion, but you can do better by improving your template.

Some methods we did not implement in this demo, but we would be impressed to see/hear about:
- Imaging filters could be a great preprocessing step
- Including a dynamics model for cleaning the output (ie. a Kalman filter)
- Including a probabilistic or dynamics model in the offset calculation would be even better
- Using multiple templates or adaptive templates that conform to variances in the brightness of the neuron over time

### Useful code
Below are some basic functions that you are free to use or build upon.

In [ ]:
from numpy.fft import fft2, ifft2, fftshift
from numpy.ma import conjugate
import scipy.optimize

def phase_corr2D(im1, im2):
    """
    Computes 2D phase correlation between two 2D arrays like images. Array shapes must match. Rich Hakim 2019
    
    Args:
        im1 (np.ndarray): array (same shape as im2)
        im2 (np.ndarray): array (same shape as im1)
    
    Returns:
        r (np.ndarray): array where each pixel is the correlation between the two arrays at that spatial offset
    """
    
    R = fft2(im1) * conjugate(fft2(im2))
    R /= np.absolute(R)
    r = fftshift(ifft2(R).real)
    return r

def register_arrays_2D(array_moving , array_ref):
    """
    Finds maximum correlation and offset (in pixels) between two 2D arrays like images. Rich Hakim 2019
    
    Args:
        array_moving (np.ndarray): set of arrays along dim-0. dim-1 & dim-2 must match array_ref
        array_ref (np.ndarray): single array. dim-0 & dim1 must match array_moving
    
    Returns:
        corrMax (np.ndarray): maximum correlation between two images for each comparison
        offset (np.ndarray): x and y offsets (in pixels) between two arrays for each comparison
    """
    
    corrMax = np.zeros(array_moving.shape[0])
    offset = np.zeros((array_moving.shape[0] , 2))
    for ii, im_mov in enumerate(array_moving):
        corrimg = phase_corr2D(array_ref, im_mov)
        corrMax[ii] = corrimg.max()
        offset[ii,:] = np.round(np.array(np.unravel_index(corrimg.argmax(), corrimg.shape)) - (np.array(corrimg.shape)/2 + 0.5))
    return corrMax , offset

def shift_image(X, dx, dy, padVal=0):
    '''
    Shifts an image 'X' by dx pixels in x and dy pixels in y. Replaces blank values with padVal.
    '''
    X = np.roll(X, dy, axis=0)
    X = np.roll(X, dx, axis=1)
    if dy>0:
        X[:dy, ...] = padVal
    elif dy<0:
        X[dy:, ...] = padVal
    if dx>0:
        X[:, :dx,...] = padVal
    elif dx<0:
        X[:, dx:,...] = padVal
    return X

def convolve_along_axis(array , kernel , axis , mode):
    '''
    Convolves an array with a kernel along a defined axis
    
    Args:
        array (np.ndarray): array you wish to convolve
        kernel (np.ndarray): array to be used as the convolutional kernel (see numpy.convolve documentation)
        axis (int): axis to convolve array along
        mode (str): see numpy.convolve documentation. Can be 'valid', 'same', 'full'
    '''
    return np.apply_along_axis(lambda m: np.convolve(m, kernel, mode=mode), axis=axis, arr=array)
    
def gaussian(x, mu, sig):
    '''
    A gaussian function (normalized)
    '''
    return 1/(np.sqrt(2*np.pi)*sig)*np.exp(-np.power((x - mu)/sig, 2)/2)

def gaussian_kernel_2D(center = (5, 5), image_size = (11, 11), sig = 1):
    """
    Generate a 2D or 1D gaussian kernel
    
    Args:
        center (tuple):  the mean position (X, Y) - where high value expected. 0-indexed. Make second value 0 to make 1D gaussian
        image_size (tuple): The total image size (width, height). Make second value 0 to make 1D gaussian
        sig (scalar): The sigma value of the gaussian
    
    Return:
        kernel (np.ndarray): 2D or 1D array of the gaussian kernel
    """
    x_axis = np.linspace(0, image_size[0]-1, image_size[0]) - center[0]
    y_axis = np.linspace(0, image_size[1]-1, image_size[1]) - center[1]
    xx, yy = np.meshgrid(x_axis, y_axis)
    kernel = np.exp(-0.5 * (np.square(xx) + np.square(yy)) / np.square(sig))
    return kernel

We also found the below functions to be useful. Documentation for them can be found online.

In [ ]:
## SOLUTION

## Problem 2: Blind source separation comparison

Hopefully you succeeded in making a video that no longer has any motion in it. If you'd like us to just send you an aligned video, we'd be happy to. With this newly aligned video, your goal is to now compare existing methods for dimensionality reduction and matrix factorization for use in extracting the fluorescence timecourse from neurons in our field of view. We provided a field of view which has multiple neurons in it. Extracting and demixing signals from a noisy background is a broad and interesting field with many applications: https://en.wikipedia.org/wiki/Signal_separation.

Our goal is to cleanly separate out the activity of neurons from noise. Some criteria for an ideal solution:
1. The output neural signal extracted for each neuron should be low in noise
2. The output neural signal should be demixed from other neurons but not split up into multiple factors
3. The spatial components of extracted neurons should either look like neurons (slightly donut-shaped circles) or 'negatives' of neurons (representing background flucuations)
4. Results are reproducible and/or are resistant to overfitting
5. No neurons are missed by the extraction algorithm due to mismatches in the magnitude of the source variances.

Here are several methods often used for blind-source-separation. While there are many others, each one of these is prominently used in the field for solving the problem of extracting neural regions-of-interest and timeseries. The methods complement each other in their objectives and constraints. When comparing these methods, remember how we defined our goal for what the ideal model should include. We value explanations in this section much more than implementation.
1. Principal Components Analysis
2. Independent Components Analysis
3. Factor Analysis
4. Non-negative matrix factorization

Simple implementations and explanations for each of these methods can be found in sklearn's decomposition module (https://scikit-learn.org/stable/modules/decomposition.html and https://scikit-learn.org/stable/modules/classes.html#module-sklearn.decomposition)

You are tasked with comparing and contrasting various blind-source-separation techniques and concluding on which method(s) are best suited for extracting our neural signals. Adding an implementation of those methods you reference and to illustrate your points would be nice but is not necessary if your writing is clear and understandable by non-experts. Try to cover the following topics:
- Is orthogonalization useful for our application? What are the pros and cons of having orthogonal factors?
- Is ICA's objective function appropriate here?
- Provide a short explanation providing intuition on how the output components of FA and NMF differ qualitatively from the output components of PCA and ICA?
- Which method should we ultimately use for our application? Why?

## Problem 3: optimizing an NMF model

Now let's focus in on one method, determine the optimal hyperparameters, and then perform some parameter estimation. If you have experience with these ideas, you'll find that typical methods may not be easily implemented or even appropriate here. Recall our goals listed in problem #2.

You are tasked with two things:
1. Identify hyperparameters that are important to tune. You are free to consider any hyperparameters, but the following should be discussed and possibly implemented:
    - Initialization method: how should we initialize the `W` & `H` matrices? Why?
    - Regularization: What type of regularization should be applied here? What are the dangers of over or under-regularizing our model?
2. Parameter estimation of the num of components (`n_components`).
    - How many neurons are there in the video we gave you? How do you know? Think about the biology here.
    - Can you conceive of a heuristic to prioritize finding low variance neurons over secondary modes of high variance neurons?
    - Can you conceive of a heuristic to ensure not include two modes of the same neuron?
    - Can you think of a way to reforumlate either the objective function, NMF procedure, or input data to approximate the above heuristics.
    
A thorough implementation of this problem might include the following:
- A grid search over selected hyperparameters
- A parameter search over `n_components` using either an information criterion, similarity metric, or cross-validation method

### Useful code
Below are some basic functions that you are free to use or build upon.

In [ ]:
def similarity_2(mat1 , mat2):
    '''
    This function compares the representations of two sets of vectors (columns of mat1 and columns of mat2).
    We assume that the vectors in mat1 and mat2 are similar up to a permutation.
    We therefore find the 'best' permutation that maximizes the similarity between the sets of vectors
    
    Args:
        mat1 (np.ndarray): a 2D array where the columns are vectors we wish to match with mat2
        mat2 (np.ndarray): a 2D array where the columns are vectors we wish to match with mat1
    
    Returns:
        corr_avg (double): the average correlation coefficients between matched vectors
    '''
    corr = mat1.T @ mat2
    ind1 , ind2 = scipy.optimize.linear_sum_assignment(corr, maximize=True)
    corr_matched = np.zeros(len(ind1))
    for ii in range(len(ind1)):
        corr_matched[ii] = np.corrcoef(mat1[:,ind1[ii]] , mat2[:,ind2[ii]])[0][1]
    corr_avg = np.mean(corr_matched)
    return corr_avg

In [ ]:
## SOLUTION